# توابع مربوطه در این قسمت تعریف شده است

In [1]:
from keras import datasets
import  random
# import keras
import numpy as np
from keras import models
from keras import layers
from tensorflow.keras.utils import to_categorical
from keras import optimizers
from keras.callbacks import EarlyStopping

(x_train, y_train), (x_test, y_test) = datasets.cifar10.load_data()
train_images = x_train.astype('float32') / 255
test_images = x_test.astype('float32') / 255
train_labels = to_categorical(y_train)
test_labels = to_categorical(y_test)
val_images = train_images[:10000]
partial_images = train_images[10000:]
val_labels = train_labels[:10000]
partial_labels = train_labels[10000:]


number_of_epochs = 20


def create_and_train_model(params):

    kernel_sizes_params = params[:5]
    num_filters_params = params[5:10]
    activation_functions_params = params[10:15]
    dropout_rate = params[15]
    optimizer = params[16]

    model = models.Sequential()
    input_shape = (32, 32, 3)
    num_classes = 10
    num_epochs = number_of_epochs

    model.add(layers.Conv2D(num_filters_params[0], kernel_sizes_params[0], input_shape=input_shape,
                            activation=activation_functions_params[0]))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(num_filters_params[1], kernel_sizes_params[1], activation=activation_functions_params[1]))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Conv2D(num_filters_params[2], kernel_sizes_params[2], activation=activation_functions_params[2]))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv2D(num_filters_params[3], kernel_sizes_params[3], activation=activation_functions_params[3]))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Conv2D(num_filters_params[4], kernel_sizes_params[4], activation=activation_functions_params[4]))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D(2, 2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(512, activation='relu'))
    model.add(layers.Dropout(dropout_rate))
    model.add(layers.Dense(num_classes, activation="softmax"))

    model.compile(loss="categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

    es = EarlyStopping(monitor="val_accuracy", patience=3)

    history = model.fit(partial_images, partial_labels, validation_data=(val_images, val_labels), epochs=num_epochs,callbacks=[es], verbose=0)

    return params , history.history['val_accuracy'][-1]

def tournament_selection(population, scores, k=2):
    selected = random.sample(list(zip(population, scores)), k)
    selected.sort(key=lambda x: x[1], reverse=True)
    return selected[0][0]


kernel_sizes = [(1, 1), (3, 3), (5, 5)]
num_filters = [2**i for i in range(3, 8)]
activation_functions = ["relu", "sigmoid", "tanh"]
dropout_rates = [0.2 , 0.25 , 0.3, 0.35 , 0.4 , 0.45 , 0.5]
optimizers_list = [optimizers.Adagrad(0.01), optimizers.Adam(0.01), optimizers.RMSprop(0.01), optimizers.SGD(0.01)]

def mutate(individual):
    index = random.randint(0, len(individual) - 1)

    if index < 5:
        individual[index] = random.choice(kernel_sizes)
    elif index < 10:
        individual[index] = random.choice(num_filters)
    elif index < 15:
        individual[index] = random.choice(activation_functions)
    elif index == 15:
        individual[index] = random.choice(dropout_rates)
    elif index == 16:
        individual[index] = random.choice(optimizers_list)

    return individual

def n_point_crossover(parent1, parent2):
    n_points = 5
    size = len(parent1)
    points = sorted(random.sample(range(1, size), n_points))
    points.append(size)

    child1, child2 = [], []
    last_point = 0
    for i, point in enumerate(points):
        if i % 2 == 0:
            child1.extend(parent1[last_point:point])
            child2.extend(parent2[last_point:point])
        else:
            child1.extend(parent2[last_point:point])
            child2.extend(parent1[last_point:point])
        last_point = point

    return child1, child2


def process_population(population):
    if not isinstance(population[0], list):
        population = [population]

    processed_population = []

    for individual in population:
        processed_individual = []
        for item in individual:
            if isinstance(item, optimizers.Optimizer):
                optimizer_instance = item.__class__(0.01)
                processed_individual.append(optimizer_instance)
            else:
                processed_individual.append(item)
        processed_population.append(processed_individual)

    return processed_population


def create_children(population, scores, number_of_children):
    number_of_children = number_of_children // 2
    for _ in range(number_of_children):
        # child1 , child2 تولید کن
        parent1 = tournament_selection(population, scores)
        parent2 = tournament_selection(population, scores)

        child1, child2 = n_point_crossover(parent1, parent2)

        child1 = mutate(child1)
        child2 = mutate(child2)

        child1 = process_population(child1)
        child2 = process_population(child2)
        child1, child2 = child1[0], child2[0]


        # اگر فرزند بدست آمده قابل قبول بود در فضای جمعیت قرار بگیرد
        try:
            a, b = create_and_train_model(child1)
            population.append(child1)
            scores.append(b)
        except :
            continue


        try:
            c, d = create_and_train_model(child2)
            population.append(child2)
            scores.append(d)
        except :
            continue

    return population

def survivor_selection(population, scores, num_survivors):
    paired = list(zip(population, scores))

    paired.sort(key=lambda x: x[1], reverse=True)

    survivors = paired[:num_survivors]

    to_remove = paired[num_survivors:]
    for individual, score in to_remove:
        index = population.index(individual)
        population.pop(index)
        scores.pop(index)

# اجرای الگوریتم تکاملی 

In [2]:
# initialization
kernel_sizes = [(1, 1), (3, 3), (5, 5)]
num_filters = [2**i for i in range(3, 8)]
activation_functions = ["relu", "sigmoid", "tanh"]
dropout_rates = [0.2, 0.25, 0.3, 0.35, 0.4, 0.45, 0.5]
optimizers_list = [
    optimizers.Adagrad(0.01),
    optimizers.Adam(0.01),
    optimizers.RMSprop(0.01),
    optimizers.SGD(0.01)
]

def generate_random_hyperparameters():
    n_k = random.choice(num_filters)
    return [
        random.choice(kernel_sizes),
        random.choice(kernel_sizes),
        random.choice(kernel_sizes),
        random.choice(kernel_sizes),
        random.choice(kernel_sizes),
        n_k,
        n_k,
        n_k * 2 if n_k < 128 else n_k ,
        n_k * 2 if n_k < 128 else n_k ,
        n_k * 4 if n_k < 64 else n_k * 2 ,
        random.choice(activation_functions),
        random.choice(activation_functions),
        random.choice(activation_functions),
        random.choice(activation_functions),
        random.choice(activation_functions),
        random.choice(dropout_rates),
        random.choice(optimizers_list)
    ]
hyperparameters = [generate_random_hyperparameters() for _ in range(30)]


#Example for chromosome [(5, 5), (3, 3), (3, 3), (5, 5), (3, 3), 2**4, 2**4, 2**6, 2**7, 2**7, "relu", "sigmoid", "relu", "relu", "relu", 0.2, optimizers.Adagrad(0.01)]]
# fitness
accuracies = []
# population
indiviual = []

accuracies_history = []

for params in hyperparameters:
    # print(params)
    try:
        hyperparametr , accy = create_and_train_model(params)
        accuracies.append(accy)
        indiviual.append(params)
        accuracies_history.append(accy)
    except :
        continue

generation = 20
pop_size = 10

print(0 , '/'  , generation , )
print(accuracies)
print('mean Accuracy:' , np.mean(accuracies) ,"best Accuracy :" , np.max(accuracies))
print('______________________________________________________________________________')

for i in range(generation):
    print(i+1 , '/'  , generation )
    accuracies_history.append(accuracies)
    create_children(indiviual , accuracies , pop_size)
    survivor_selection(indiviual , accuracies , pop_size)
    print(accuracies)
    print('mean Accuracy:' , np.mean(accuracies) ,"best Accuracy :" , np.max(accuracies))
    print('______________________________________________________________________________')


print("best Hyper parameter found :" , indiviual[np.argmax(accuracies)])

C:\Users\Meta Pc\PycharmProjects\pythonProject\.venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


0 / 20
[0.18520000576972961, 0.7445999979972839, 0.7164999842643738, 0.24089999496936798]
mean Accuracy: 0.4717999957501888 best Accuracy : 0.7445999979972839
______________________________________________________________________________
1 / 20
[0.7445999979972839, 0.7164999842643738, 0.6830999851226807, 0.7217000126838684, 0.7113999724388123, 0.7196999788284302, 0.7042999863624573, 0.713100016117096, 0.6978999972343445, 0.6965000033378601]
mean Accuracy: 0.7108799934387207 best Accuracy : 0.7445999979972839
______________________________________________________________________________
2 / 20
[0.7445999979972839, 0.7164999842643738, 0.7217000126838684, 0.7196999788284302, 0.7156999707221985, 0.7317000031471252, 0.7278000116348267, 0.7193999886512756, 0.7196999788284302, 0.7276999950408936]
mean Accuracy: 0.7244499921798706 best Accuracy : 0.7445999979972839
______________________________________________________________________________
3 / 20
[0.7445999979972839, 0.7317000031471252, 0.7

# قسمت دوم سوال 2

یکی از اصلی ترین محدودیت هایی که در این مسئله وجود دارد این است که برای اینکه وسیع تر بتوانیم جستجو کنیم نیاز به یک سخت افزار قوی داریم چون هر بار اجرا کردن و آموزش دادن شبکه زمان نسبتا زیادی میبرد و نمیتوانیم خیلی تعداد اعضای جمعیت و هم چنین تعداد تولید فرزندان را افزایش دهیم . مورد دیگری که مشکل ساز میشود این است که ساختار شبکه را نمیتوانیم تغییر دهیم به این معنا که در صورت سوال خواسته شده ات که فقط تعداد کرنل ها و سایز کرنل ها تغییر داده شود و مجاز نیستیم که به عنوان مثال تعداد لایه ها را از 5 لایه به 8 لایه تغییر دهیم در این صورت هم قسمتی از فضای جستجو را از دست میدهیم .یکی دیگر از چالش هایی که ممکن است مشکل ساز باشد این است که سایز تصاویر کوچک است و این مورد اجازه نمیدهد از فیلتر های با سایز بزرگتر استفاده کنیم و فقط محدود به استفاده کردن از فیلتر های کوچک هستیم . یک مورد دیگر هم که میتواند کمک کننده باشد که در این مسئله از آن استفاده نشد این است که تعداد لایه ها را بزرگ بگیریم و از اسکیپ کانکشن در بین لایه ها استفاده کنیم که دچار مشکلات محو گرادیان نشویم و مدل قدرتمند تری بتوانیم بسازیم . هم چنین در بین کلاس هایی که در این دیتاست داریم کلاس ماشین و کامیون و کلاس سگ و گربه نسبتا شبیه به یکدیگر هستند و ممکن است در تشخیص این موارد از یکدیگر به مشکل بخوریم که برای حل کردن این مشکلات میتوان از تکنیک های دیتا آگمنتیشن استفاده کرد . یعنی برای اینکه شبکه بتواند این موارد را از هم بهتر تشخیص دهد داده های موجود در این کلاس ها را با تکنیک هایی مانند روتیت کردن یا تغییر کنتراستن تصویر افزایش دهیم تا شبکه تقویت شود .  